In [57]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import sys

In [58]:
def preprocess(classes, features):
   # Read the dataset
   data = pd.read_csv('./dataset/dry_bean_dataset.csv')
   # Filter rows based on the 'Class' column
   # Perform linear interpolation for missing values in the 'MinorAxisLength' column
   data[features] = data[features].copy().fillna(data[features].mean())
   # Manually perform Min-Max scaling
   for column in features:
      min_val = data[column].min()
      max_val = data[column].max()
      data[column] = (data[column] - min_val) / (max_val - min_val)

   # Shuffle the data
   # data = data.sample(frac=1, random_state=0).reset_index(drop=True)
   data = shuffle(data, random_state=0)
   data.insert(0, 'bias', 1)
   features.append("bias")
   X = data[features].values
   Y = np.where(data['Class'] == classes[0], -1, np.where(data['Class']==classes[1],0,1))
   x_train, x_test, y_train, y_test = train_test_split(
      X, Y, test_size=0.4, random_state=42)
   return x_train, x_test, y_train, y_test

In [59]:
x_train,x_test,y_train,y_test = preprocess(["BOMBAY","CALI","SIRA"],["Area","Perimeter","MajorAxisLength","MinorAxisLength","roundnes"])

x_train.shape

(90, 6)

In [60]:
def sigmoidFunc(x):
    return (1/(1+np.exp(-x)))
def derivative_sigmoid(x):
    return x* (1-x)

In [61]:
def initialize_weights(hiddenlayers,n_neurons):
   weights = [np.random.rand(n_neurons[i], n_neurons[i+1]) for i in range(hiddenlayers+1)]
   return weights

In [62]:
weights = initialize_weights(2,[6,3,4,3])

weights

[array([[0.04046884, 0.0880038 , 0.84884163],
        [0.71756034, 0.21689564, 0.3425878 ],
        [0.70171023, 0.25164503, 0.00772788],
        [0.90233507, 0.28643799, 0.50696501],
        [0.76340322, 0.23922198, 0.68113253],
        [0.50379039, 0.66171777, 0.64960866]]),
 array([[0.22804778, 0.19996604, 0.7908184 , 0.94893144],
        [0.61272973, 0.66836565, 0.86576866, 0.21044338],
        [0.9970884 , 0.34250179, 0.81412574, 0.7735016 ]]),
 array([[0.83576367, 0.49455961, 0.58305995],
        [0.94214406, 0.5636021 , 0.74829389],
        [0.37668208, 0.8815399 , 0.27010214],
        [0.11905739, 0.77083384, 0.78200676]])]

In [63]:
# Calculate neuron activation for an input
def activate_input(weights, inputs):
    activation = 0
    for i in range(len(weights.T)):
        for j in range(len(weights)):
            activation += weights[j] * inputs[i][j]
    activation=sigmoidFunc(activation)
    return activation

In [64]:
def activate_hidden(weights, activatedInput):
    activation = 0
    for i in range(len(weights)):
        for j in range(len(weights)):
            activation += weights[j] * activatedInput[i]
    activation=sigmoidFunc(activation)
    return activation

In [67]:
hidden_layers = [3,4]
cnt = 0

for count in range(len(x_train)):
    new_input = activate_input(weights[0], x_train)

    for i in range(1, len(weights)):
        new_input = activate_hidden(weights[i], new_input)

        if i == len(hidden_layers):
            mx = -1 * sys.float_info.max
            idx = -1

            for j in range(len(new_input)):
                if new_input[j] > mx:
                    mx = new_input[j]
                    idx = j
                # Assuming y_train is a NumPy array
                value_to_find = y_train[count]
                index_array = np.where(y_train == value_to_find)[0]

                if index_array.size > 0:
                    true_class_index = index_array[0]

                    if true_class_index == idx:
                        cnt += 1
                else:
                    print(f"Error: {value_to_find} not found in y_train array")
            

print("Training acc= ", cnt / len(y_train))

Training acc=  1.0555555555555556


In [10]:
def backprop_out(y_train,activations):
    for i in y_train:
        output_layer_errors = activations[-1] - i
        delta = output_layer_errors * derivative_sigmoid(output_layer_errors)
    return delta

In [11]:
def backprop_hidden(delta,act):
    print(act)
    for j in range(len(act)-1):
        hidden_layer_error = np.dot(delta[j],act)
        hidden_delta = hidden_layer_error * derivative_sigmoid(hidden_layer_error)
    return hidden_delta

In [12]:
Deltas =[]
delta = backprop_out(y_train,activations)
Deltas.append(delta)
activation =activations[:len(activations)-1]
for i in activation[::-1]:
    delta =backprop_hidden(delta,i)
    Deltas.append(delta)
Deltas

[0.98642612 0.9959615  0.997238   0.8236099 ]
[0.99927757 0.99937132 0.99986972]


[array([0.00023716, 0.00169066, 0.00142503]),
 array([1.97317945e-06, 2.01148433e-06, 2.01664009e-06, 1.37588290e-06]),
 array([4.04021720e-12, 4.04097531e-12, 4.04500687e-12])]

In [13]:
def update_weights_out(weights, deltas, activations, learning_rate):
    new_weight = []
    deltas = deltas[::-1]
    for i in range(len(weights)):
            for j in range(len(weights[i])):
                dotprod = np.dot(deltas[j],activations)
                mult = learning_rate * dotprod
                plus = weights[i][j]+mult
                new_weight.append(plus)
    return new_weight

In [14]:
weights_updated_out=[]
for i in range(len(weights)):
    weights_updated_out.append(update_weights_out(weights[i],Deltas[i],activations[i],0.01))
weights_updated_out

[[array([0.76806025, 0.76806025, 0.76806025]),
  array([0.32527292, 0.32527293, 0.32527293]),
  array([0.32102562, 0.32102562, 0.32102562]),
  array([0.30153553, 0.30153553, 0.30153554]),
  array([0.56325436, 0.56325436, 0.56325437]),
  array([0.87752977, 0.87752977, 0.87752977]),
  array([0.29560358, 0.29560358, 0.29560359]),
  array([0.82630571, 0.82630571, 0.82630572]),
  array([0.79034871, 0.79034871, 0.79034871]),
  array([0.92331953, 0.92331954, 0.92331954]),
  array([0.57696267, 0.57696267, 0.57696268]),
  array([0.55112383, 0.55112383, 0.55112383]),
  array([0.27695572, 0.27695572, 0.27695572]),
  array([0.52622308, 0.52622309, 0.52622309]),
  array([0.64825111, 0.64825111, 0.64825111]),
  array([0.8095263, 0.8095263, 0.8095263]),
  array([0.65942928, 0.65942928, 0.65942929]),
  array([0.94034262, 0.94034262, 0.94034262])],
 [array([0.42010627, 0.42010627, 0.42010627, 0.42010626]),
  array([0.72863838, 0.72863838, 0.72863838, 0.72863838]),
  array([0.59267671, 0.59267671, 0.592